# Matrix Inversion

This notebook is for exercise purposes only. Solutions to the problems will be released in week 9.

Here we will look at how to write a matrix inversion routine. This is a classic problem in numerical & computational methods. In a future exercise, we'll look at similar routines that are available in SciPy.


## Simultaneous Equations

Many problems in physics require solving simultaneous equations. When these become large and complex, numerical routines are required.

A set of simultaneous equations can always be written in matrix form, for example, two equations in two unknowns ($x_1$ and $x_2$)

$$ax_1 + bx_2 = y_1$$
$$cx_1 + dx_2 = y_2$$

can be rewritten as

$$\left(\begin{array}{cc} a & b \\ c & d\end{array}\right)
\left(\begin{array}{c} x_1 \\ x_2 \end{array}\right) = 
\left(\begin{array}{c} y_1 \\ y_2 \end{array}\right)$$

An arbitrary set of equations is

$$Ax = y$$

where A is the matrix of coefficients, x is the vector of unknown variables $x_1$, $x_2$, ...
and b is the known vector of constants. The solution to Equation 1 can be obtained
by pre-multiplying by the inverse of A:

$$A^{-1} A x = A^{-1} y$$

giving :

$$x = A^{-1} y$$

## Finding the Inverse
There are many ways to find the inverse of a matrix.  Here we will use Cramer's rule for a general square matrix :

$$A^{-1} = \frac{1}{\det{A}}C^T$$

where C is the matrix of cofactors of A (or equivalently $C^T$ is the adjugate of A).

If we know the order of the input matrix in advance, we could do the algebra to find a formula in terms of the matrix elements. Eg for a $2x2$ matrix we could write :

In [8]:
import numpy as np
def det2x2(m):
    if m.shape[0] != 2 or m.shape[1] != 2:
        raise Exception("Non2x2Matrix")
    
    return ( m[0][0]*m[1][1] ) - ( m[0][1]*m[1][0] )

def inverse2x2(m):
    if m.shape[0] != 2 or m.shape[1] != 2:
        raise Exception("Non2x2Matrix")

    det = det2x2(m)
    tmp = np.empty([2,2])
    tmp[0][0] = m[1][1] / det
    tmp[0][1] = -1 * m[0][1] / det
    tmp[1][0] = -1 * m[1][0] / det
    tmp[1][1] = m[0][0] / det
    return tmp

(Note that the first thing we do, in both functions, is to check that the user really has given us a $2x2$ matrix...)

We could expand this to $3x3$ and $4x4$ matrices, but clearly that is going to get complicated (and error prone) pretty quickly.

Instead, if we had methods to calculate the deteriminant and adjugate of a general square matrix, we could just do something like :

In [9]:
def inverse(m):
    d = det(m)        
    c = adjugate(m)
    return np.true_divide(c,d)

In order to write these functions, it is worth noting that both the determinant and adjugate of a general matrix depend on sub-matrices that are of order $n-1$.  This is easily seen in the Laplace formula for the determinant of a $3 \times 3$ matrix :

\begin{equation*}
\begin{vmatrix}
a && b && c \\
d && e && f \\
g && h && i
\end{vmatrix}
=
a \begin{vmatrix}
e && f \\
h && i
\end{vmatrix}
-b \begin{vmatrix}
d && f \\
g && i
\end{vmatrix}
+
c \begin{vmatrix}
d && e \\
g && h
\end{vmatrix}
\end{equation*}

So before writing the det() and adjugate() functions, we write a function to find the $(i,j)^{th}$ sub-matrix of a general square matrix.

In [10]:
def submatrix(m,i,j):
    if not (m.shape[0] == m.shape[1]):
        raise Exception("NonSquareMatrix")
    
    # create a new matrix
    n = m.shape[0]-1
    tmp = np.empty([n,n])
    
    # get the indices of the matrix that will be used
    tmpi = list(range(m.shape[0]))
    del tmpi[i]
    tmpj = list(range(m.shape[1]))
    del tmpj[j]
    
    for ii in range(n):
        for jj in range(n):
            tmp[ii][jj] = m[tmpi[ii]][tmpj[jj]]

    return tmp

Again, the first thing we do is check that the argument given to the function is suitable for further processing and bail out if it isn't.

Next we can write the function that will calculate the determinant (using Laplace's formula) and adjugate of a general square matrix.

In [11]:
import math
def det(m):
    if not (m.shape[0] == m.shape[1]):
        raise Exception("NonSquareMatrix")
    
    if (m.shape[0]==2):
        return det2x2(m)
        
    else:
        tmp = 0
        # use 0-th row to calculate determinant
        for j in range(m.shape[0]):
            tmp = tmp + (math.pow(-1,j) * m[0][j] * det(submatrix(m,0,j)))
    
    return tmp

def adjugate(m):
    if not (m.shape[0] == m.shape[1]):
        raise Exception("NonSquareMatrix")
    
    tmp = np.empty(m.shape)
    
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            # note the order of indices below !
            tmp[j][i] = (math.pow(-1,i+j) * det(submatrix(m,i,j)))
    return tmp

Note that the determinant uses **recursion**, i.e. it makes calls to _itself_. This is an extremely powerful concept in programming and numerical methods.

Although it is not without limitation :
https://stackoverflow.com/questions/3323001/what-is-the-maximum-recursion-depth-in-python-and-how-to-increase-it


Now we have everything we need for the inverse function. (This time adding some protection against non-square matrices). 

In [12]:
def inverse(m):
    if not (m.shape[0] == m.shape[1]):
        raise Exception("NonSquareMatrix")
    
    if m.shape[0]==2:
        return inverse2x2(m)
    
    d = det(m)        
    c = adjugate(m)
    return np.true_divide(c,d)

We can test this function with a simple matrix :

In [14]:
a = np.array([[1,2,3],[1,0,0],[7,8,9]])
print(inverse(a))

import scipy.linalg
print(scipy.linalg.inv(a))

[[ 0.          1.          0.        ]
 [-1.5        -2.          0.5       ]
 [ 1.33333333  1.         -0.33333333]]
[[-0.          1.          0.        ]
 [-1.5        -2.          0.5       ]
 [ 1.33333333  1.         -0.33333333]]


But how do we know the answer is correct ?  We could calculate it by hand, say, and check the result given by the function is the same :

In [15]:
a = np.array([[1,2,3],[1,0,0],[7,8,9]])
inv_a = inverse(a)

test = np.array([[0.,1.,0.],[-1.5,-2.,0.5],[1.33333333,1.,-0.33333333]])
assert ( np.allclose( test, inv_a, atol=1e-19) )

Note the use of numpy.allclose() here, which allows us to avoid sensitivity to rounding errors.  What happens if you try this using np.array_equal() instead ?

However, we've only tested one particular case here.  Can you write a more general test function ?